<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br><br><br>

# Listed Volatility and Variance Derivatives

**Wiley Finance (2017)**

Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | http://books.tpq.io

<img src="http://hilpisch.com/../images/lvvd_cover.png" alt="Derivatives Analytics with Python" width="30%" align="left" border="0">

# VSTOXX Index

## Introduction

This chapter is about the (re-) calculation of the VSTOXX index, the volatility index based on EURO STOXX 50 index options. The goal is to achieve a good understanding of the processes and underlying mechanics of calculating the VSTOXX index. You will find all background information as well as Python code that will enable you to recalculate historical VSTOXX index values as well as current ones in (almost) real-time. The chapter about the _Model-Free Replication of Variance_ provides the theoretical background for the concepts presented in this chapter.

The (main) VSTOXX index itself is based on two sub-indexes, which themselves are derived from Eurex option series for both European puts and calls on the EURO STOXX 50 index. The algorithm, and therefore this chapter as well, is comprised of three main parts:

* collect and clean-up the data of the necessary option series
* compute the sub-indexes from the option data
* compute the VSTOXX index from the relevant sub-indexes

A few remarks about the option series and sub-indexes used and their expiry dates and time horizons, respectively, seem in order. There are eight sub-indexes of the VSTOXX which each measure the implied volatility of an option series with fixed expiry. For example, the VSTOXX 1M sub-index starts with the option series that has one month expiry and is calculated up to two days prior to the fixed maturity date of the according option series. The VSTOXX index measures the implied volatility of an "imaginary" options series with a fixed time to expiry of 30 days. This is achieved through linear interpolation of the two nearest sub-indexes, generally VSTOXX 1M and VSTOXX 2M. On the two days before VSTOXX 1M expiry, the VSTOXX 2M and VSTOXX 3M are used instead and an extrapolation takes place.

The following table lists all the sub-indexes and provides additional information.

    ============ ======= ============== ===========================================
     Sub-index    Code    ISIN           Settlement date of the option series used
    ============ ======= ============== ===========================================
     VSTOXX 1M    V6I1    DE000A0G87B2   The last available within 1 month
     VSTOXX 2M    V6I2    DE000A0G87C0   The last available within 2 months
     VSTOXX 3M    V6I3    DE000A0G87D8   The last available within 3 months
     VSTOXX 6M    V6I4    DE000A0G87E6   The last available within 6 months
     VSTOXX 9M    V6I5    DE000A0G87F3   The last available within 9 months
     VSTOXX 12M   V6I6    DE000A0G87G1   The last available within 12 months
     VSTOXX 18M   V6I7    DE000A0G87H9   The last available within 18 months
     VSTOXX 24M   V6I8    DE000A0G87J5   The last available within 24 months
    ============ ======= ============== ===========================================

## Collecting Option Data

As pointed out, the VSTOXX is based on two sub-indexes, generally the VSTOXX 1M and VSTOXX 2M, sometimes VSTOXX 2M and VSTOXX 3M. The sub-indexes themselves are based on the option series on the EURO STOXX 50 index with respective time to expiry. We therefore need the prices of all options with maturities up to 3 months. We use historical data as provided by Eurex itself as the data source. See the web site http://bit.ly/1GY5KCI.

The code to collect the data can be found in the module ``index_collect_option_data.py`` (see appendix for the complete script). Like usual, the module starts with some imports and parameter definitions.

In [ ]:
!sed -n 11,24p scripts/index_collect_option_data.py

In addition, the module contains six functions. The first is ``collect_option_series()``:

In [ ]:
import sys; sys.path.append('./scripts/')

In [ ]:
import index_collect_option_data as icod

In [ ]:
icod.collect_option_series??

This function collects the data of the option series with maturity in the month ``month`` and year ``year``. It is called by the ``function start_collecting()`` and calls the ``function get_data()`` for every single day from the date ``start`` to today. It returns a complete set of prices (both puts and calls) for that series.

The second function is ``get_data()``.

In [ ]:
icod.get_data??

This one is called by the function ``collect_option_series()`` and calls itself the functions ``get_data_from_www()``, ``parse_data(data, date)`` and ``merge_and_filter()``. It returns the prices of the option series with expiry date in month ``month`` and year ``year`` for the day ``date``.

The third function is ``get_data_from_www()``.

In [ ]:
icod.get_data_from_www??

The function collects the prices of an option series for a single day (defined by ``date``) from the web. The option series is defined by the date of its expiry, given by ``matMonth`` and ``matYear``, the type of the options is given by ``oType`` which can be either ``Put`` or ``Call``. It returns a complete HTML file.

``merge_and_filter()`` is the fourth function.

In [ ]:
icod.merge_and_filter??

This one gets two time series ``puts`` and ``calls`` (typically of the same option series), merges them, filters out all options with prices below 0.5 and returns the resulting pandas ``DataFrame`` object.

``parse_data()`` is the fifth function.

In [ ]:
icod.parse_data??

It gets the string ``data`` which contains the HTML text delivered by function ``get_data_from_www()``, parses that string to a pandas ``DataFrame`` object with double index ``date`` and ``strike price`` and returns that object.

The sixth and final function is ``data_collection()``.

In [ ]:
icod.data_collection??

This function is to initiate and finalize the collection of all relevant option series data sets. It saves the resulting data in a file named ``index_option_series.h5``.

In [ ]:
path = './data/'

Let us collect option data since all other steps depend on this data. We import the module as ``icod``.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings; warnings.simplefilter('ignore')

Next, fix a target day relative to today such that you hit a business day for which closing data is available.

In [ ]:
today = dt.datetime.now()
## make sure to hit a business day
target_day = today - dt.timedelta(days=2)
ds = target_day.strftime('%Y%m%d')
ds

In [ ]:
URL = 'https://www.eurex.com/ex-en/data/statistics/market-statistics-online/'
URL += '100!onlineStats?productGroupId=13370&productId=69660&viewType=3&'
URL += 'cp=%s&month=%s&year=%s&busDate=%s'
URL % ('Call', 12, 2020, '20201111')

Then, for example, collect option data for puts and calls with a maturity as defined by the parameters as follows.

In [ ]:
## adjust maturity parameters if necessary
call_data = icod.get_data_from_www(oType='Call', matMonth=12,
                                   matYear=2020, date=ds)
put_data = icod.get_data_from_www(oType='Put', matMonth=12,
                                  matYear=2020, date=ds)

The return objects need to be parsed.

In [ ]:
## parse the raw data
calls = icod.parse_data(call_data, target_day)
puts = icod.parse_data(put_data, target_day)

Let us have a look at some meta information about the call options data.

In [ ]:
calls.info()

In [ ]:
calls.head()

And about the put options data.

In [ ]:
puts.info()

In a next step, we take out the daily settlement prices for both the puts and calls and define two new ``DataFrame`` objects.

In [ ]:
calls = pd.DataFrame(calls.rename(
        columns={'Daily settlem. price': 'Call_Price'}
                            ).pop('Call_Price').astype(float))

In [ ]:
puts = pd.DataFrame(puts.rename(
        columns={'Daily settlem. price': 'Put_Price'}
                            ).pop('Put_Price').astype(float))

These two get then merged via the function ``merge_and_filter()`` into another new ``DataFrame`` object.

In [ ]:
dataset = icod.merge_and_filter(puts, calls)
dataset.info()

This whole procedure is implemented in the function ``collect_option_series()`` which yields the same result.

In [ ]:
os = icod.collect_option_series(12, 2020, target_day)
os.info()

The function ``data_collection()`` repeats this procedure for all those dates for which option data is available and writes (appends) the results in a HDF5 database file.

In [ ]:
# uncomment to initiate the process (takes a while)
# %time icod.data_collection(path)

For the further analyses, we open this HDF5 database file.

In [ ]:
store = pd.HDFStore(path + 'index_option_series.h5', 'r')
store

The collected option series data is easily read from the HDF5 database file in monthly chunks.

In [ ]:
Dec20 = store['Dec20']
Dec20.info()
store.close()

Some selected option prices from the large data set:

In [ ]:
Dec20.iloc[25:35]

## Calculating the Sub-Indexes

In this section, we use the data file created in the previous one. For all dates of the data file, the Python module ``index_subindex_calculation.py`` (see the appendix for the complete script) used in this section decides whether the VSTOXX 1M sub-index is defined or not (remember that the sub-index is not defined at the final settlement day and one day before). If it is defined, the script computes the value of the sub-indexes VSTOXX 1M and VSTOXX 2M; if not, it computes the values of the sub-indexes VSTOXX 2M and VSTOXX 3M, respectively. Finally, it returns a pandas ``DataFrame`` object with the three time series.

### The Algorithm

First, we focus on the computation of the value of a single sub-index for a given date. Given are the prices $C_i, i \in \{0,...,n\},$ of a series of  European call options on the EURO STOXX 50 with fixed maturity date $T$ and exercise prices $K_i, i \in \{0,...,n\},$ as well as the prices $P_i, i \in \{0,...,n\},$ of a series of European put options on EURO STOXX 50 with the same maturity date $T$ and exercise prices $K_i$. Let further hold $K_i < K_{i+1}$ for all $i \in \{0,....,n-1\}$.

Then, the value of the relevant sub-index $V$ is as follows (see also the chapter about _Model-Free Replication of Variance_):

$$
V = 100 \cdot \sqrt{\hat{\sigma}^2}
$$

with

$$
\hat{\sigma}^2 = \frac{2}{T} \sum_{i=0}^n \frac{\Delta K_i}{{K_i}^2} \mathrm{e}^{rT} M_i - \frac{1}{T}\left( \frac{F}{K_*}-1\right)^2
$$

where

$$
\begin{array}{ll}
 \Delta K_i &=\left\{ \begin{array}{ll} K_1-K_0 & \mbox{for } i=0 \\  \dfrac{K_{i+1}-K_{i-1}}{2} &  \mbox{for } i = 1,...,n-1 \\   K_n-K_{n-1} & \mbox{for } i=n \end{array} \right. \\  \\
 r          &=  \mbox{constant risk-free short rate appropriate for maturity $T$} \\ \\
 F 	    &= K_j+ \mathrm{e}^{rT}|C_j-P_j|, \mbox{ where } j=\displaystyle \min_{i \in \{0,...,n\}}\{|C_i-P_i|\} \\ \\
 K_*        &= \displaystyle \max_{ K_{i | i \in \{0,...,n\}}} \{K_i < F \}, \\ \\
 M_i        & =    \left\{ \begin{array}{ll} P_i & \mbox{for } K_i<K_* \\  \dfrac{P_i-C_i}{2} &  \mbox{for } K_i=K_*\\   C_i & \mbox{for } K_i>K_* \end{array} \right.
\end{array}
$$

We implement a function to compute one value of a single sub-index. Thereafter, we extend that function to compute time series for both VSTOXX 1M and VSTOXX 2M indexes as well as parts of the VSTOXX 3M index. Imports again make up the beginning of the script.

In [ ]:
!sed -n 10,15p scripts/index_subindex_calculation.py

A core function of the script is ``compute_subindex()``.

In [ ]:
import index_subindex_calculation as isc

In [ ]:
isc.compute_subindex??

This script calculates a single index value. It implements mainly the following steps:

* the calculation of $\Delta K_i$
* the computation of the forward price and the index of $K_*$
* the selection of the at-the-money option and the out-of-the-money options
* the combination of the results of the other three steps

The next step is the derivation of time series data for the VSTOXX 1M and VSTOXX 2M as well as parts of VSTOXX 3M indexes and storage of the results in a pandas ``DataFrame`` object. As data source we use the file created in the last section. Remember, that this file contains a dictionary-like ``HDFStore`` object with one entry for every options series. The keys for the entries are three letter abbreviations of the respective month's name plus the actual year represented by two numbers, for example ``Mar16``, ``Jun16``, ``Dec20`` and so on. The value of an entry is a pandas ``DataFrame`` object with a pandas ``MultiIndex`` (date, strike price) and prices for the put and call options for the dates and strike prices.

All this is implemented as function ``make_subindex()``.

In [ ]:
isc.make_subindex??

This function uses the collected option series data and selects those data sub-sets needed for the calculation at hand. It generates sub-index values for all those days for which option data is available. The result is a pandas ``DataFrame`` object.

Let us see how it works. To this end, we first import the module as ``isc``.

In [ ]:
import index_subindex_calculation as isc
si = isc.make_subindex(path)
si

For comparison, we retrieve the "real" historical VSTOXX (sub-) index values.

In [ ]:
url = 'https://hilpisch.com/vstoxx_eikon_eod_data.csv'

In [ ]:
vs = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
vs.head()

Next, combine the re-calculated VSTOXX 2M values with the historical ones into a new ``DataFrame`` object and add a new column with the absolute differences.

In [ ]:
comp = pd.concat((si['V6I2'], vs['.V6I2']),
          axis=1, join='inner')
comp.index = comp.index.normalize()
comp.columns = ['CALC', 'REAL']
comp['DIFF'] = comp['CALC'] - comp['REAL']
comp

The following figure shows the two time series in direct comparison.

In [ ]:
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'  # set serif font

In [ ]:
comp[['CALC', 'REAL']].plot(style=['ro', 'b'], figsize=(10, 6));

<p style="font-family: monospace;">Calculated VSTOXX 2M sub-index values vs. real ones.

The following figure shows the point-wise differences between the two time series.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(comp.index, comp['DIFF'])
plt.gcf().autofmt_xdate();

<p style="font-family: monospace;">Differences of calculated VSTOXX 2M index values and real ones.

## Calculating the VSTOXX Index

If the values for the sub-indexes VSTOXX 1M and VSTOXX 2M, $V_1$ and $V_2$ say, are given, then the value for the VSTOXX index itself, $V$ say, is calculated by the linear interpolation of $V_1$ and $V_2$:

$$
V = \sqrt{\left(T_1\cdot V_1^2\cdot\left(\frac{N_{T_{2}}-N_{30}}{N_{T_{2}}-N_{T_1}}\right)+T_{2}\cdot V_2^2\cdot\left(\frac{N_{30}-N_{T_1}}{N_{T_{2}}-N_{T_{1}}}\right)\right)\cdot\frac{N_{365}}{N_{30}}}
$$

where

* $N_{T_1}=$ time to expiry of $V_1$´s options series in seconds
* $N_{T_2}=$ time to expiry of $V_2$´s options series in seconds
* $N_{30}= 30$ days in seconds
* $N_{365}=$ time for a standard year in seconds
* $T_1= N_{T_1}/N_{365}$
* $T_2= N_{T_2}/N_{365}$

Recall that the sub-index VSTOXX 1M is not defined on the final settlement day of the underlying option series and the day before. For these dates, we use VSTOXX 2M and VSTOXX 3M as $V_1$ and $V_2$, respectively.

The Python module ``index_vstoxx_calculation.py`` (see the appendix for the module in its entirety) implements the VSTOXX index calculation routine &mdash; given the respective sub-index time series data sets. The module starts like usual with some imports.

In [ ]:
!sed -n 9,12p scripts/index_vstoxx_calculation.py

The function ``calculate_vstoxx()`` is the core of the module.

In [ ]:
import index_vstoxx_calculation as ivc

In [ ]:
ivc.calculate_vstoxx??

As its single argument, the function takes the path to a CSV file containing historical VSTOXX data for the index itself and the sub-indexes. The re-calculation of it then is as straightforward as follows.

In [ ]:
import index_vstoxx_calculation as ivc
%time data = ivc.calculate_vstoxx(url)

Let us inspect the pandas ``DataFrame`` with the results.

In [ ]:
data.info()

A brief look at the absolute average error of the re-calculation reveals that the implementation yields quite accurate results.

In [ ]:
## output: average error of re-calculation
data['Difference'].mean()

The following figure compares the original `.V2TX` time series with the re-calculated values.

In [ ]:
## original vs. re-calculated VSTOXX index
data[['.V2TX', 'VSTOXX']].plot(subplots=True, figsize=(10, 6),
                              style="blue", grid=True);

In [ ]:
## original vs. re-calculated VSTOXX index
data[['.V2TX', 'VSTOXX']].plot(figsize=(10, 6), style=['-', '.']);

<p style="font-family: monospace;">Historical VSTOXX index values re-calculated vs. real ones.

Finally, the following figure presents the absolute differences. The figure shows that the differences are in general marginal with a few outliers observed here and there.

In [ ]:
## differences between single values
data['Difference'].plot(figsize=(10, 6), style="r", grid=True,
                        ylim=(-1, 1));

<p style="font-family: monospace;">Differences of historical VSTOXX index values re-calculated vs. real ones.

## Conclusions

This chapter (re-) calculates the  VSTOXX volatility index based on historical sub-index values and based on the volatility index definition as derived in the chapter _Model-Free Replication of Variance_. The chapter also shows how to calculate the sub-index values themselves based on EURO STOXX 50 options data. Python code is provided to automatically collect such data from the Eurex web site.

## Python Scripts

### ``index_collect_option_data.py``

In [ ]:
!cat scripts/index_collect_option_data.py

### `index_subindex_calculation.py`

In [ ]:
!cat scripts/index_subindex_calculation.py

### `index_vstoxx_calculation.py`

In [ ]:
!cat scripts/index_vstoxx_calculation.py

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>